In [ ]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import warnings
from privacygan import privacy_gan as pg
from privacygan.lfw import lfw_gan
from datetime import datetime
from sklearn.datasets import fetch_lfw_people

warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__, "device:", device)


### Load dataset

In [ ]:
# Load LFW dataset via sklearn and build train/test splits
lfw = fetch_lfw_people(min_faces_per_person=20, resize=0.5)
X_all = lfw.data.astype(np.float32)
X_all = (X_all - 127.5) / 127.5

frac = 0.1
n = int(frac * len(X_all))
idx = np.random.permutation(len(X_all))
X = X_all[idx[:n]]
X_comp = X_all[idx[n:]]

print('training set size:', X.shape)
print('test set size:', X_comp.shape)


### Simple GAN

In [ ]:
(generator, discriminator, dLosses, gLosses) = pg.SimpGAN(
    X,
    generator=lfw_gan.LFW_Generator(),
    discriminator=lfw_gan.LFW_Discriminator(),
    epochs=1,
    batchSize=128,
)


In [ ]:
#perform white box attack
Acc = pg.WBattack(X,X_comp, discriminator)

In [ ]:
with torch.no_grad():
    scores_train = discriminator(torch.tensor(X, dtype=torch.float32, device=device)).cpu().numpy().squeeze()
    scores_test = discriminator(torch.tensor(X_comp, dtype=torch.float32, device=device)).cpu().numpy().squeeze()

plt.hist(scores_train, color='r', alpha=0.5, label='train', density=True, bins=50)
plt.hist(scores_test, color='b', alpha=0.5, label='test', density=True, bins=50)
plt.xlabel('Discriminator probability')
plt.ylabel('Normed frequency')
plt.title('GAN')
plt.legend()


In [ ]:
#generate synthetic images
noise = np.random.normal(0, 1, size=[X.shape[0], 100])
generatedImages = generator.predict(noise)
temp = generatedImages[:25].reshape(25, 62, 47)
plt.figure(figsize=(5, 5))
for i in range(temp.shape[0]):
    plt.subplot(5,5, i+1)
    plt.imshow(temp[i], interpolation='nearest', cmap='gray_r')
    plt.axis('off')
plt.tight_layout()

### Private GAN

In [ ]:
### Test privGAN (with n_reps = 2)
generators = [lfw_gan.LFW_Generator(), lfw_gan.LFW_Generator()]
discriminators = [lfw_gan.LFW_Discriminator(), lfw_gan.LFW_Discriminator()]
pDisc = lfw_gan.LFW_DiscriminatorPrivate(OutSize=2)

(generators, discriminators, _, dLosses, dpLosses, gLosses) = pg.privGAN(
    X,
    epochs=1,
    disc_epochs=1,
    batchSize=128,
    generators=generators,
    discriminators=discriminators,
    pDisc=pDisc,
    privacy_ratio=1.0,
)


In [ ]:
#perform white box attack
pg.WBattack_priv(X,X_comp, discriminators)

In [ ]:
with torch.no_grad():
    scores_train = discriminators[0](torch.tensor(X, dtype=torch.float32, device=device)).cpu().numpy().squeeze()
    scores_test = discriminators[0](torch.tensor(X_comp, dtype=torch.float32, device=device)).cpu().numpy().squeeze()

plt.hist(scores_train, color='r', alpha=0.5, label='train', density=True, bins=50)
plt.hist(scores_test, color='b', alpha=0.5, label='test', density=True, bins=50)
plt.xlabel('Discriminator probability')
plt.ylabel('Normed frequency')
plt.title('privGAN')
plt.legend()


In [ ]:
#plot distribution of discriminator scores of training and test set
plt.hist(discriminators[0].predict(X)[:,0],color = 'r', alpha = 0.5, label = 'train', density = True, bins = 50)
plt.hist(discriminators[0].predict(X_comp)[:,0],color = 'b', alpha = 0.5, label = 'test', density = True, bins = 50)
plt.xlabel('Discriminator probability')
plt.ylabel('Normed frequency')
plt.title('privGAN')
plt.legend()